In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
import statistics

%matplotlib inline

pd.set_option("max_columns", None)
pd.options.display.float_format = '{:,.10f}'.format

# Data understading

First we load the available databases for Boston.
Despite that there is more recent information available in inside Airbnb, our analysis uses the data extraction from 13 February in order to avoid data after the massive spread of COVID 19 thoughout USA.

In [ ]:
path='C:/Users/rwester/Documents/Ricardo W/Apuntes,Cursos,Estudios/Data Science Nanodegree programm/Boston/'
listings=pd.read_csv(path+'listings.csv')
calendar=pd.read_csv(path+'calendar.csv')
reviews=pd.read_csv(path+'reviews.csv')

Interesting are the offered price and availability by date for every listing

In [ ]:
calendar.info()
calendar.head()

The Calendar shows the availability for the next 365 days with their respective price.
The price by date could be usefull to determine the price variations and their amount during the year.
There is information about 3903 listings

In [ ]:
print(calendar.date.min(),calendar.date.max())
print(calendar.listing_id.nunique())

Mean availability (due to other guest reservation or block of the day by the host) are less than half for the next 365 days.

In [ ]:
calendar['available'].value_counts()

The listings database seems to be very rich in information. 
Neighborhood, GPS location, score and amenities description are available for every listing

In [ ]:
listings.info()
listings.head()

Reviews by guests for every listing

In [ ]:
reviews.info()
print(reviews.head())
print(reviews['date'].min())
print(reviews['date'].max())

# Business Understanding

Base on the available data seeing above, we ask ourself the following questions: 

* a)Which are the most demanded seasons in boston?
* b)How much rise prices during high season?
* c)Which are the most expensive neighbourhoods?
* d)Which are the most demanded neighbourhoods?

For answering these questions we will use data from the calendar, reviews and listings databases.

## Which are the most demanded seasons in boston?

To answer the first question, we need to estimate the occupancy by date.

The Calendar information may not be useful cause the reason behind an unavailable date is not public available, 
(it's imposible to distinguish wheter it's already booked by other guest or the date is block by the host).

Instead, the assumption of the occupancy model from the "Inside Airbnb San Francisco Model" will be used. 

The assumption states that 50% of the total listing's bookings write a review 
(source: http://insideairbnb.com/about.html#disclaimers).

We plot the bookings by date and highlight the dates of the boston marathon from the last 3 years, event that gathers more than
30.000 competitors by year and that with certainty causes a peak of demand that day. (source: https://www.baa.org/races/boston-marathon/results/participation)

In [ ]:
review_new = reviews.copy()

review_new = review_new.groupby('date')['id'].unique().reset_index()
review_new['id']=review_new['id'].map(lambda x: 2*len(x))
review_new['date'] = pd.to_datetime(review_new['date'])

d1=dt.datetime(2017,1,1)
bm17=dt.datetime(2017,4,17)
bm18=dt.datetime(2018,4,16)
bm19=dt.datetime(2019,4,15)

nye17=dt.datetime(2016,12,31)
nye18=dt.datetime(2017,12,31)
nye19=dt.datetime(2018,12,31)
nye20=dt.datetime(2019,12,31)

review_new=review_new[review_new['date']>=d1]

events={'event':['bm17','bm18','bm19','nye17','nye18','nye19','nye20']
        ,'date':[bm17,bm18,bm19,nye17,nye18,nye19,nye20]
        ,'color':['red','red','red','green','green','green','green']
        }

events_df=pd.DataFrame(events)

plt.figure(figsize=(15, 5))
plt.plot(review_new['date'], review_new['id'])

for i in range(0,events_df.shape[0]):
    plt.axvline(x=events_df.iloc[i,1],c=events_df.iloc[i,2],ls=':')

plt.title('Boston estimated occupancy by day')
plt.ylabel('n° of estimated reservations')
plt.savefig('boston_estimated_occupancy.png')
plt.show();

It can be seen that there is a local maximum of demand near the date of the marathon.
However, it must be considered that the date of the review is not necesary the date of the stay in the listing place.

If we see in detail the estimated reservation by day one week before and after the boston marathon, it can be estimate an average delay between the start day of occupancy and the review. 

In the following plots the boston marathon date is resalted in red. With a high certainty it can be assume that the observed 
peak of bookigns actually occurred the day before the competition (competitors need to arrive at least one day before the 
marathon ir order to be properly restful before the competition), so we can conclude that users write a review on average two 
days after their stay. This aproximation will be use forward for determine the most demanded day in the week.

In [ ]:
def bm_subplot(data,events):
    '''
    Function that display subplots with the estimated bookings by day considering a -/+ 1 week interval near the boston marathon dates
    dates from the last 3 years.
    Input: data: dataframe with the estimated bookings
           events: dataframe with last 3 marathon version's dates
    Output: display 3 subplots
    '''
    dw=dt.timedelta(days=7)
    
    plt.figure(figsize=(15,5))
    
    for i in range(0,3):
        #marathon date from the last 3 years
        md=events.iloc[i,1]
        #marathon date interval -/+ 1 week
        mi=data[(data['date']>=(md-dw)) & (data['date']<(md+dw))]
        
        plt.subplot(2,2,i+1)
        plt.plot(mi['date'], mi['id'])
        plt.axvline(x=md,c='red',ls='-.')
        plt.title('Estimated bookings by day near '+str(2017+i)+' BM')
        plt.ylabel('n° listings occupancy')
        plt.xticks([])
        plt.grid(True)
        
    return plt.show()

bm_subplot(review_new,events_df)

Now that we estimated the average delay we analize the occupancy by month. 

It is quite obvious that the demand is higher during the warmer months during Spring and Summer, but the question is:
how much rise demand during that period?

By plotting the estimated demand by month from the previous 3 years, it can be seen that months between May and October
are the most demanded. After October the demands falls through Autumm achieving its lower value during the 
hardest part of winter (January and February)

In [ ]:
review_new.index = review_new['date']
review_monthly=review_new.resample('M').sum()

def rw_barplot(size,data,plot_var,title,ylabel,savefig_name):

    plt.figure(figsize=(size[0], size[1]))
    x_ticks=pd.DataFrame(data=data.index)
    x_ticks['date']=x_ticks['date'].dt.strftime("%b %Y")
    x_pos = np.arange(len(x_ticks))

    plt.bar(x_pos, data[plot_var],align='center')
    plt.xticks(x_pos, x_ticks['date'],rotation=90)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.savefig(savefig_name)
    return plt.show()

              
rw_barplot([15,5],review_monthly,'id','Monthly estimated occupancy','n° of estimated reservations','boston_monthly_occupancy.png')             
              
              
review_monthly['month_name']=review_monthly.index.strftime("%b")
high_season=['May','Jun','Jul','Aug','Sep','Oct']
low_season=['Nov','Dec','Jan','Feb','Mar','Apr']
lowest_season=['Jan','Feb']

seasons=[high_season,low_season,lowest_season]
col_name=['high_season_demand','low_season_demand','lowest_season_demand']

#we exclude partially data from february 2020
review_monthly=review_monthly.iloc[0:-1,:]

def seasons_statistics(data,seasons,col_mame):
    '''
    Function that estimates the average value of target variable in data for the months specified in the lists inside seasons.
    Input: data: dataframe with monthly records for the target variable.
           seasons: different list of months that defines partitions of the year.
           col_name: name of the output columns
    Output: 2 column dataframe with the season or partition of year name and mean value for the target variable.
    '''
    stats=[]
    
    for j in range(0,len(seasons)):
        season_data=[data.iloc[i][0] for i in range(data.shape[0]) if data.iloc[i][1] in seasons[j]]
        stats.append(statistics.mean(season_data))
    
    dic_aux={'period':col_name,
          'mean_value':stats}
    output=pd.DataFrame(dic_aux)
    return output

sd=seasons_statistics(review_monthly,seasons,col_name)

def summary_statement(sdf):
    avg_high=sdf.iloc[0,1]
    avg_low=sdf.iloc[1,1]
    avg_lowest=sdf.iloc[2,1]
    aux1="Average monthly reservations during high season are: " + str(avg_high) +".\n"
    aux2="Average monthly reservations during low season are: " + str(avg_low)+ " and demand decreases "
    aux3=str((avg_low-avg_high)/avg_high)+ " vs high season.\n"+"Average monthly reservations during January and February are: "
    aux4=str(avg_lowest)+" and demand decreases "+str((avg_lowest-avg_high)/avg_high)+ " vs high season."
    return aux1+aux2+aux3+aux4


print(summary_statement(sd))

## How much rise prices during high season?

In [ ]:
def cast_monetary_float(columns,df):
    '''
    Function that cast columns with monetary data from a string type to float
    Input: columns: column list with columns names to be cast
           df: dataframe with target columns
    Output: dataframe with transform columns
    '''
    for col in columns:
        try:
            df[col]=df[col].str.replace(',', '')
            df[col]=df[col].str.replace('$', '')
            df[col]=df[col].astype(float)
        except:
            print('Formato incorrecto')

        return df

Data preprocessing from calendar database: monetary columns are transform to float.     

In [ ]:
calendar=cast_monetary_float(['price','adjusted_price'],calendar)
calendar_new=calendar.copy()

calendar_new['date'] = pd.to_datetime(calendar_new['date'])
calendar_new=calendar_new[['date','price']]

Price information correspond to the one available in the published calendar of every listing, therefore these prices may be 
subject to future changes, but still work for identifying price tendencies.

Monthly average prices show us that the most demanded months are also the expensivest ones.

The average price of the most require months(May-Oct) is 21% higher than 
the average of the lowest demanded (Nov-Apr) and if we compare the lowest and highest months average (Feb-20 and May-20) the 
difference is from $85,8 (52pp) , that's a tremendous increase in just 3 months!

In [ ]:
calendar_new.index = calendar_new['date']
monthly_prices=calendar_new.resample('M').mean()

rw_barplot([15,7],monthly_prices,'price','Monthly listed prices','average price','boston_monthly_listed_prices.png')             
              
monthly_prices['month_name']=monthly_prices.index.strftime("%b")
columns=['high_season_price','low_season_price','lowest_season_price']

sp=seasons_statistics(monthly_prices,seasons,col_name)

avg_high=sp.iloc[0,1]
avg_low=sp.iloc[1,1]
avg_lowest=sp.iloc[2,1]

print('Average listing price during high season are:', avg_high, '.')
print('Average listing price during low season are:', avg_low, 'and high season prices are',(avg_high-avg_low)/avg_low, 'higher')
print('Average listing price during January and February are:', avg_lowest, 'and high season prices are',(avg_high-avg_lowest)/avg_lowest, 'higher')

greatest_dif=monthly_prices.loc[dt.datetime(2020,5,31)]['price']-monthly_prices.loc[dt.datetime(2020,2,29)]['price']
print(greatest_dif)
monthly_prices


There are also differences between days of week. Sunday to Tuesday are the cheapest days, from wednesday prices start to 
rise and continue ascending till Saturday, the expensivest day. Saturday could be on average 5% expensiver than Monday or 
Tuesday. The price increase logically has a direct relationship with demand rise during weekends. The later can be confirmed
with the estimated demand by days of week.

In [ ]:
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

calendar['date'] = pd.to_datetime(calendar['date'])
calendar['dayofweek'] = calendar.date.dt.weekday

calendar['day_name']=[cats[i] for i in calendar['dayofweek']]
price_week=calendar[['day_name','price']]
price_week = price_week.groupby(price_week['day_name'])['price'].mean().reindex(cats)

def rw_linear_plot(data,label):
    data.plot()
    ticks = list(range(0, 7, 1))
    labels = label.split()
    plt.xticks(ticks, labels)
    return plt.plot;

rw_linear_plot(price_week,"Mon Tues Weds Thurs Fri Sat Sun")

price_week

Next we apreciate the average number of bookings by day corrected by the estimated delay of two days. It can be seen that 
weekend's Days are the most demanded headed by Friday, which makes sence cause people that travel for leisure try to take the 
most of weekend.

In [ ]:
l3y=dt.datetime(2017,2,13)

review_day = reviews.copy()
review_day['date'] = pd.to_datetime(review_day['date'])
review_day=review_day[review_day['date']>=l3y]

cats = ['Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
cats_order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

review_day = review_day.groupby('date')['id'].unique().reset_index()
review_day['n_reservation']=review_day['id'].map(lambda x: 2*len(x))

review_day['dayofweek']=review_day.date.dt.weekday
review_day['day_name']=[cats[i] for i in review_day['dayofweek']]

occup_week=review_day[['day_name','n_reservation']]

occup_week = occup_week.groupby(occup_week['day_name'])['n_reservation'].mean().reindex(cats_order)

rw_linear_plot(occup_week,"Mon Tues Weds Thurs Fri Sat Sun")

# Which are the most expensive neighbourhoods?

In [ ]:
reviews.head()

Now, proceed to estimate the expensivest neighbourhoods of boston
First, we check the listing´s price distribution:

In [ ]:
listings=cast_monetary_float(['price'],listings)
print(listings['price'].describe())

#clearly there exists outlyers in the base
fig1, ax1 = plt.subplots()
ax1.set_title('Price distribution')
ax1.boxplot(listings['price'])

If we inspect the most expensive listings we realize that there are not available (perhaps it was a joke or fake by someone) or
the listed price is much lower than the one that appears below.

In [ ]:
listings.sort_values(by='price', axis=0, ascending=False).head()

From the main indicators seen above, we have to define a border above which the listings prices will be considered outlyers
800 dolars seems to be a plausible limit, because with this criteria only 46 of the 3903 listings will be discarded.

In [ ]:
data=listings.loc[listings.price>800]['price']
print(data.count())

fig1, ax1 = plt.subplots()
ax1.set_title('Price distribution')
ax1.boxplot(listings.loc[listings.price<=800]['price'])

Finally we obtain average prices by neighbourhood.

The South Boston Waterfront and the West end are the expensivest neighbourhoods to book in Airbnb. 

The first is a district that has suffered a series of renovations since the beginning of the 21st Century that had turn it into a flourish neigbourhood.

The second is a mixed-use commercial and residential area district near downtown.

After them in prices comes Mission Hill, that is known by his architectural value and then a series of neighbourhoods that are 
located nearby downtown, like Charlestown, back bay, Beacon hill, North End and Chinatown.

The cheapest districts are suburbs located far from downtown, like Mattapan, hyde park and dorchester.

In [ ]:
sort_order = listings.query('price <= 800')\
                    .groupby('neighbourhood_cleansed')['price']\
                    .median()\
                    .sort_values(ascending=False)\
                    .index
sns.boxplot(y='price', x='neighbourhood_cleansed', data=listings.query('price <= 800'), 
            order=sort_order)
ax = plt.gca()
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Publish prices distribution by neighbourhood')
plt.show()

## Which are the most demanded neighbourhoods?

In order to answer question four we proceed to elaborate a series of indicators 

The number of reservation per ce don't tell us so much cause bigger neighbourhoods tend to have more listings offer, like 
Dorchester, East Boston and Roxbury.

Therefore we need to look at an indicator at a listing level, like the mean occupancy by listing

In [ ]:
def demand_subplots(listings):
    '''
    Function that display a 2x2 subplot grid with a series of calculated variables at a district level.
    Input: listings: dataframe with the listings data
    Output: mean prices by neightbourhood and a 2x2 subplot grid
    '''
    color=['orange','green','red','#1f77b4']
    col=['reviews_per_month','id','reviews_per_month','price']
    x_title=['avg. monthly bookings','number of listings','number of bookings','price ($USD)']
    title=['Monthly Bookings by neighbourhood','Number of listings','Average Monthly Bookings by listing'
           ,'Average price by neighbourhood']
    
    for i in range(0,4):
        if i==0 or i==2:
            if i==0:
                sp_data=listings.groupby(by='neighbourhood_cleansed').sum()[[col[i]]].sort_values(by=col[i], ascending=False)
            else:
                sp_data=listings.groupby(by='neighbourhood_cleansed').mean()[[col[i]]].sort_values(by=col[i], ascending=False)
            
            sp_data[col[i]]=sp_data[col[i]]*2
            sp_data=sp_data.round({col[i]:1})
        
        elif i==1:
            sp_data=listings.groupby(by='neighbourhood_cleansed').count()[[col[i]]].sort_values(by=col[i], ascending=False)
            
        elif i==3:
            sp_data=listings[listings.price<=800].groupby(by='neighbourhood_cleansed').mean()[[col[i]]].sort_values(by=col[i]
                    , ascending=False)
        
        plt.subplot(2,2,i+1)
        sp_data[col[i]].plot(kind = 'barh' , figsize = (12,10),color=color[i])
        plt.ylabel('')
        plt.xlabel('avg. monthly bookings');
        plt.title(title[i]);
        plt.grid(True)

    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.55)

    print(sp_data)
    
    return plt.show()

demand_subplots(listings)

Surprisingly, the most demanded places are not the best for sightseeing as one had expected.

On the one hand Dorchester and Allston, first and third most requested districts, are both very inexpensive. 
(Dorchester average price is almost half of downtown). Despite Dorchester is very far from 
Downtown, Red subway line operates throug it, offering a good transport option. 
Allston has the advantage that is not far from downtown offering a convenient option for tourist travelling by car.

On the other hand, East Boston and South End (second and fourth respectively) are middle price neightbouhoods not so far from
downtown and with a very broad offer of public transport.